# End-to-end Fuzzy Deduplication

GPU accelerated implementation of a MinHash-LSH based fuzzy deduplication. For more information about semantic deduplication in NeMo Curator, refer to the [Deduplication](https://docs.nvidia.com/nemo/curator/latest/curate-text/process-data/deduplication/index.html) section of the documentation page.

The tutorial here shows how to run Fuzzy Duplication on text data by executing a 2 end to end workflows which does the following:

1. Read original dataset
2. Compute MinHashes signatures of these documents
3. Perform LSH - Group Minhashes into bands/buckets and shuffle these bands/buckets so that documents in the same bucket are in the same batch/file.
4. Convert the LSH outputs (bucket_id -> doc_id mapping) into a edgelist in preperation for connected components. 
5. Compute connected components across all potential duplicates found via LSH.
6. Generate list of duplicate documents by randomly selecting 1 document to keep from each group/component and dropping the rest.
7. Remove duplicates based on the generated duplicate list.

We also allow users to also run these steps independently, which will be covered in the step by step tutorial in the same directory as this tutorial.

In [1]:
import os

# Silence Curator logs via Loguru
os.environ["LOGURU_LEVEL"] = "ERROR"

import pandas as pd

input_path = os.path.abspath("./input")  # Path to input dataset
fuzzy_output_dir = os.path.abspath("./output/e2e")  # Path to store all fuzzy outputs including cache & deduped dataset
cache_path = os.path.join(
    fuzzy_output_dir, "cache"
)  # Path to store fuzzy deduplication intermediates (minhash, lsh etc.)
deduplicated_output_path = os.path.join(fuzzy_output_dir, "fuzzy_deduped_dataset")

input_filetype = (
    "parquet"  # this can be either of jsonl or parquet (you'll need to change how input data is generated)
)
output_filetype = "parquet"  # this can be either of jsonl or parquet

In [2]:
from nemo_curator.utils.file_utils import get_all_file_paths_under

if len(get_all_file_paths_under(input_path)) == 0:
    import os
    import uuid

    from datasets import load_dataset

    input_df = load_dataset("roneneldan/TinyStories", split="train").to_pandas()
    num_rows_per_file = 10_000

    os.makedirs(input_path, exist_ok=True)

    for i, start_idx in enumerate(range(0, len(input_df), num_rows_per_file)):
        end_idx = min(len(input_df), start_idx + num_rows_per_file)
        subset_df = input_df.iloc[start_idx:end_idx].copy()
        subset_df["id"] = [str(uuid.uuid4()) for _ in range(len(subset_df))]
        subset_df.to_parquet(os.path.join(input_path, f"part_{i}.parquet"), index=False)

    print(f"Created {len(os.listdir(input_path))} files")

Created 212 files


## Running as a Single Stage (End-to-End)

See the [API Reference](https://docs.nvidia.com/nemo/curator/latest/apidocs/stages/stages.deduplication.fuzzy.workflow.html#api) for more information about the `FuzzyDeduplicationWorkflow` class.

### General Notes
#### ID Generation
1. The Fuzzy Deduplication Workflow doesn't utilize any existing IDs in the input dataset and instead generates IDs on the fly using an ID Generator actor.
2. The ID Generator gives each row a unique increasing integer ID, based on the order files are read.
3. This avoids expensive ID->Integer encoding for the underlying connected components algorithm which only supports integer IDs.
4. When we find duplicates, we save these integer IDs in sorted files with multiple row groups.
5. We also save a `fuzzy_id_generator.json` which maintains a mapping of input file partitions to ID ranges for that batch.
6. During removal, reading the same file groups will give the same integer IDs, using the min/max ID values, we can find all corresponding duplicates in that range making the process faster.

#### Performance Considerations
1. LSH - Configuring bands_per_iteration controls how many bands to process simultanesouly in a single shuffle. Higher values can lead to faster performance but might increase memory pressure.
2. A low `input_blocksize` may not saturate the GPUs enough while a high `input_blocksize` can lead to OOM errors during MinHash and excessive object store usage during removal. It's recommend to keep it at 1-1.5GiB and reduce if running into OOMs during MinHash.
3. The removal step can be memory intensive and it's recommend to set a higher fraction of object store memory for removal (if the machine has enough RAM). The `RayDataExecutor` showed better results during duplicate removal.
4. The removal workflow is CPU only and can be run  on machines that don't have GPUs

#### Hyperparameter Considerations
1. The current defaults for Fuzzy deduplication (260 hashes, 13 hashes per band) approximate finding documents with a jaccard similarity of 0.8. For more information on selecting the number of bands/hashes it's recommended to analyze the S curve and tolerable threshold for false postives (and negatives). More information about LSH can be found in section `3.4.2` [here](http://infolab.stanford.edu/~ullman/mmds/ch3n.pdf).
2. The `char_ngrams` values of 24 is set to approximate roughly ngrams that correspond to ~5 words.


In [3]:
from nemo_curator.stages.deduplication.fuzzy import FuzzyDeduplicationWorkflow
from nemo_curator.stages.deduplication.id_generator import CURATOR_DEDUP_ID_STR
from nemo_curator.stages.text.deduplication import TextDuplicatesRemovalWorkflow

identification_workflow = FuzzyDeduplicationWorkflow(
    cache_path=cache_path,
    output_path=fuzzy_output_dir,
    input_path=input_path,
    input_filetype=input_filetype,
    input_blocksize="1GiB",
    text_field="text",
    seed=42,
    char_ngrams=24,
    minhashes_per_band=13,
    bands_per_iteration=10,
)

removal_workflow = TextDuplicatesRemovalWorkflow(
    input_path=input_path,
    ids_to_remove_path=os.path.join(fuzzy_output_dir, "FuzzyDuplicateIds"),
    output_path=deduplicated_output_path,
    input_filetype=input_filetype,
    input_blocksize="1GiB",
    ids_to_remove_duplicate_id_field=CURATOR_DEDUP_ID_STR,
    id_generator_path=os.path.join(fuzzy_output_dir, "fuzzy_id_generator.json"),
    output_filetype="parquet",
)

In [4]:
from nemo_curator.backends.experimental.ray_data import RayDataExecutor
from nemo_curator.core.client import RayClient

client = RayClient(num_cpus=64, num_gpus=2)  # change as needed
client.start()

_ = identification_workflow.run()
_ = removal_workflow.run(executor=RayDataExecutor())

2025-11-18 14:07:41,489	INFO worker.py:1692 -- Using address 127.0.1.1:6379 set in the environment variable RAY_ADDRESS
2025-11-18 14:07:41,491	INFO worker.py:1833 -- Connecting to existing Ray cluster at address: 127.0.1.1:6379...
[2025-11-18 14:07:46,761 W 2783477 2783477] global_state_accessor.cc:505: Some processes that the driver needs to connect to have not registered with GCS, so retrying. Have you run 'ray start' on this node?
[2025-11-18 14:07:47,763 W 2783477 2783477] global_state_accessor.cc:505: Some processes that the driver needs to connect to have not registered with GCS, so retrying. Have you run 'ray start' on this node?
[2025-11-18 14:07:48,764 W 2783477 2783477] global_state_accessor.cc:505: Some processes that the driver needs to connect to have not registered with GCS, so retrying. Have you run 'ray start' on this node?


2025-11-18 14:07:43,899	INFO usage_lib.py:447 -- Usage stats collection is disabled.
2025-11-18 14:07:43,899	INFO scripts.py:914 -- Local node IP: 127.0.1.1
2025-11-18 14:07:48,884	SUCC scripts.py:950 -- --------------------
2025-11-18 14:07:48,884	SUCC scripts.py:951 -- Ray runtime started.
2025-11-18 14:07:48,884	SUCC scripts.py:952 -- --------------------
2025-11-18 14:07:48,884	INFO scripts.py:954 -- Next steps
2025-11-18 14:07:48,885	INFO scripts.py:957 -- To add another node to this Ray cluster, run
2025-11-18 14:07:48,885	INFO scripts.py:960 --   ray start --address='127.0.1.1:6379'
2025-11-18 14:07:48,885	INFO scripts.py:969 -- To connect to this Ray cluster:
2025-11-18 14:07:48,885	INFO scripts.py:971 -- import ray
2025-11-18 14:07:48,885	INFO scripts.py:972 -- ray.init(_node_ip_address='127.0.1.1')
2025-11-18 14:07:48,885	INFO scripts.py:984 -- To submit a Ray job using the Ray Jobs CLI:
2025-11-18 14:07:48,885	INFO scripts.py:985 --   RAY_API_SERVER_ADDRESS='http://127.0.0.1

[2025-11-18 14:07:49,765 I 2783477 2783477] global_state_accessor.cc:487: This node has an IP address of 127.0.1.1, but we cannot find a local Raylet with the same address. This can happen when you connect to the Ray cluster with a different IP address or when connecting to a container.
2025-11-18 14:07:49,767	INFO worker.py:2013 -- Connected to Ray cluster.
/raid/adattagupta/NeMo-Curator/.venv/lib/python3.12/site-packages/ray/_private/worker.py:2052: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(
2025-11-18 14:07:54,429	INFO worker.py:1692 -- Using address 127.0.1.1:6379 set in the environment variable RAY_ADDRESS
2025-11-18 14:07:54,430	INFO worker.py:1833 -- Connecting to existing Ray cluster at address: 127.0.1.1:6379...
2025-11-18 14:07:54,436	INFO worker.py:2004 

Running 0: 0.00 row [00:00, ? row/s]

[2025-11-18 14:09:31,674 E 2783477 2783477] core_worker.cc:2153: Actor with class name: 'MapWorker(MapBatches(ParquetReaderStageActor))' and ID: 'fc35e5fe28e1db1b3d820d5e06000000' has constructor arguments in the object store and max_restarts > 0. If the arguments in the object store go out of scope or are lost, the actor restart will fail. See https://github.com/ray-project/ray/issues/53727 for more details.


- MapBatches(FilePartitioningStageTask) 1: 0.00 row [00:00, ? row/s]

- StreamingRepartition 2: 0.00 row [00:00, ? row/s]

- MapBatches(ParquetReaderStageActor) 3: 0.00 row [00:00, ? row/s]

- MapBatches(TextDuplicatesRemovalStageTask)->MapBatches(ParquetWriterTask) 4: 0.00 row [00:00, ? row/s]

2025-11-18 14:09:31,699	WARNING resource_manager.py:134 -- ⚠️  Ray's object store is configured to use only 10.4% of available memory (186.3GiB out of 1786.3GiB total). For optimal Ray Data performance, we recommend setting the object store to at least 50% of available memory. You can do this by setting the 'object_store_memory' parameter when calling ray.init() or by setting the RAY_DEFAULT_OBJECT_STORE_MEMORY_PROPORTION environment variable.
2025-11-18 14:09:57,466	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_5_0 execution finished in 25.83 seconds
2025-11-18 14:09:57,538	INFO worker.py:1692 -- Using address 127.0.1.1:6379 set in the environment variable RAY_ADDRESS
2025-11-18 14:09:57,540	INFO worker.py:1833 -- Connecting to existing Ray cluster at address: 127.0.1.1:6379...
2025-11-18 14:09:57,546	INFO worker.py:2004 -- Connected to Ray cluster. View the dashboard at http://127.0.0.1:8265 


### Looking at Intermediate Results and Output

#### MinHash Results
1. `_curator_dedup_id` - The IDs assigned to this dataset on the fly during the intial read.
2. `_minhash_signature` - MinHash Signature

#### LSH Results
1. `_bucket_id` - The bucket/band identifier
2. `_curator_dedup_id` - List of all document IDs that belong to that bucket

#### Buckets To Edges Result
1. `_curator_dedup_id_x`, `_curator_dedup_id_y` - Mapping of edges in a Graph where each column are documents that are potential duplicates.

In [5]:
minhash_path = os.path.join(cache_path, "MinHashStage")
display(pd.read_parquet(os.path.join(minhash_path, os.listdir(minhash_path)[0])).head())

lsh_path = os.path.join(cache_path, "LSHStage")
display(pd.read_parquet(os.path.join(lsh_path, os.listdir(lsh_path)[0])).head())

b2e_path = os.path.join(cache_path, "BucketsToEdgesStage")
display(pd.read_parquet(os.path.join(b2e_path, os.listdir(b2e_path)[0])).head())

,_curator_dedup_id,_minhash_signature
0,0,"[11644717, 429172, 6014805, 86354, 2387151, 49..."
1,1,"[2103321, 653305, 2941429, 5780991, 6977799, 7..."
2,2,"[1891498, 3797631, 2961751, 50078, 21382505, 5..."
3,3,"[1286357, 4060996, 1376561, 3044837, 7369355, ..."
4,4,"[6272013, 12535265, 819579, 5975720, 25677928,..."


,_bucket_id,_curator_dedup_id
0,b0_000055fd7daae1e46223e8b7e06bf2e0,"[1158375, 2079489]"
1,b0_00006a5f30f7b2c96588bfc1bfb5321a,"[365218, 1933514]"
2,b0_00006f316d5bd251bd83702e3f1e017f,"[161590, 771961]"
3,b0_0000d90e9e4140a7ac31e6b227a62f62,"[8290, 567169]"
4,b0_0000f975e5bcda25838df43b0d37737f,"[965853, 1334885]"


,_curator_dedup_id_x,_curator_dedup_id_y
0,1158375,2079489
1,365218,1933514
2,161590,771961
3,8290,567169
4,965853,1334885


#### Connected Components Result

1. `_curator_dedup_id` - The document IDs
2. `_duplicate_group_id` - The group ID that document belongs to. Documents with the same duplicate group ID are duplicates

In [6]:
cc_path = os.path.join(cache_path, "ConnectedComponentsStage")
cc_df = pd.read_parquet(cc_path)  # works with pandas since the input here is small
display(cc_df)
grouped_cc_df = cc_df.groupby("_duplicate_group_id")._curator_dedup_id.agg(list)
display(grouped_cc_df)
display(cc_df._duplicate_group_id.value_counts())

,_curator_dedup_id,_duplicate_group_id
0,580,482048
1,582,320014
2,588,320015
3,590,482054
4,592,482056
...,...,...
640509,2119663,479764
640510,2119665,159145
640511,2119668,320010
640512,2119672,320011


_duplicate_group_id
1              [197441, 577]
4              [197445, 581]
6              [197448, 584]
7              [197449, 585]
10             [197453, 589]
                 ...        
640504    [2119660, 1942709]
640506    [1942713, 2119664]
640507    [2119666, 1942715]
640510    [1942719, 2119670]
640513    [2119675, 1942724]
Name: _curator_dedup_id, Length: 320043, dtype: object

_duplicate_group_id
42263     230
335749      3
534437      3
399046      3
534446      3
         ... 
238792      2
426811      2
426810      2
426809      2
316339      2
Name: count, Length: 320043, dtype: int64

Based on the distribution above we can see that there is one cluster/group where 230 documents are all duplicates followed by many smaller clusters with 2/3 documents that are duplicates.

#### FuzzyDuplicateIds Results (List of duplicate docs to remove)
1. `_curator_dedup_id` - ID of docs in the removal list

In [7]:
duplicate_ids_path = os.path.join(fuzzy_output_dir, "FuzzyDuplicateIds")
duplicates_df = pd.read_parquet(duplicate_ids_path)
display(duplicates_df.head())

print(f"Number of duplicate documents found for removal: {len(duplicates_df)}")

,_curator_dedup_id
0,576
1,579
2,584
3,585
4,591


Number of duplicate documents found for removal: 320471


#### Checking that the duplicate ids list contains only one document per group

In [9]:
# As an example let's look at the largest cluster corresponding to _duplicate_group_id `42263`
duplicate_group = grouped_cc_df.loc[42263]

# number of docs in the removal list from this group
docs_to_remove_in_group = duplicates_df._curator_dedup_id.isin(duplicate_group).sum()

print(f"Number of documents in the duplicate group: {len(duplicate_group)}")
print(f"Number of documents in the removal list from the same group: {docs_to_remove_in_group}")
assert docs_to_remove_in_group == (len(duplicate_group) - 1)  # noqa: S101

Number of documents in the duplicate group: 230
Number of documents in the removal list from the same group: 229


#### Advanced: Looking at examples of duplicate documents

1. This analysis involves re-reading the input data with the same ID mapping that was used during duplicate identification.
2. Merging the input data with the connected components results on the `_curator_dedup_id` column to associate each document which the duplicate group it belongs to which can be used for further analysis.

NOTE: This analsis approach is itended for smaller datasets and only works for cases where the connected components dataframe is small and fits comfortable in memory. It is not recommended for larger datasets.

In [10]:
from nemo_curator.pipeline import Pipeline
from nemo_curator.stages.base import ProcessingStage
from nemo_curator.stages.resources import Resources
from nemo_curator.stages.text.io.reader import ParquetReader
from nemo_curator.tasks.document import DocumentBatch


class CustomMergeStage(ProcessingStage[DocumentBatch, DocumentBatch]):
    """
    Warning: This should not be attempted with large connected components results.
    A small stage that merges the input data (using the id's generated) with the connected components result.
    Works because CC results are small enough to fit per batch.
    """

    _resources = Resources(cpus=1.0)

    def process(self, batch: DocumentBatch) -> DocumentBatch:
        df = batch.to_pandas().merge(cc_df, how="inner", on=[CURATOR_DEDUP_ID_STR])
        return DocumentBatch(
            task_id=batch.task_id, dataset_name=batch.dataset_name, data=df, _stage_perf=batch._stage_perf
        )


pipeline = Pipeline(
    name="Explore duplicates",
    stages=[ParquetReader(file_paths=input_path, blocksize="1GiB", _assign_ids=True), CustomMergeStage()],
)

In [11]:
from nemo_curator.core.client import RayClient
from nemo_curator.stages.deduplication.id_generator import create_id_generator_actor, kill_id_generator_actor

client = RayClient(num_cpus=8)  # change as needed
client.start()

create_id_generator_actor(filepath=os.path.join(fuzzy_output_dir, "fuzzy_id_generator.json"))
merged_results = pipeline.run()
merged_df = pd.concat([batch.to_pandas() for batch in merged_results]).sort_values("_duplicate_group_id")
kill_id_generator_actor()

2025-11-18 14:10:28,579	INFO worker.py:1692 -- Using address 127.0.1.1:6379 set in the environment variable RAY_ADDRESS
2025-11-18 14:10:28,581	INFO worker.py:1833 -- Connecting to existing Ray cluster at address: 127.0.1.1:6379...
2025-11-18 14:10:28,581	INFO worker.py:1851 -- Calling ray.init() again after it has already been called.
2025-11-18 14:10:29,244	INFO worker.py:1692 -- Using address 127.0.1.1:6379 set in the environment variable RAY_ADDRESS
2025-11-18 14:10:29,246	INFO worker.py:1833 -- Connecting to existing Ray cluster at address: 127.0.1.1:6379...
2025-11-18 14:10:29,252	INFO worker.py:2004 -- Connected to Ray cluster. View the dashboard at http://127.0.0.1:8265 
2025-11-18 14:10:29,266	INFO worker.py:1692 -- Using address 127.0.1.1:6379 set in the environment variable RAY_ADDRESS
2025-11-18 14:10:29,267	INFO worker.py:1833 -- Connecting to existing Ray cluster at address: 127.0.1.1:6379...
2025-11-18 14:10:29,268	INFO worker.py:1851 -- Calling ray.init() again after it

In [12]:
display(merged_df[merged_df._curator_dedup_id.isin(duplicate_group)])

,text,id,_curator_dedup_id,_duplicate_group_id
115055,,2c41b5de-fa15-4cd4-b802-68524a1ce611,387189,42263
433730,,d928378b-b4e5-4795-97a1-584ae258df71,1437273,42263
556435,,59011818-a780-41eb-97b0-abfa08a074f5,1852286,42263
556434,,0cee9ce9-f5ce-4d6a-bf0e-bb9c14f9d972,1852285,42263
496191,,57be64cd-f1af-4974-9034-9fbd3375e5e9,1657722,42263
...,...,...,...,...
369284,,7c2f5029-2c35-496f-a885-92a51e92177d,1214999,42263
369285,,dee264f7-a172-467d-866a-a596a3753be8,1215000,42263
587520,,b03db266-da2d-425b-abda-bc59c30c24a3,1955797,42263
587512,,9bae741c-aa83-414d-a081-1a75b1167361,1955789,42263


The largest cluster/group of duplicates in this dataset seems to be documents with empty/no text.
Looking through the next largest cluster here are the documents.

In [13]:
duplicates = merged_df[merged_df._curator_dedup_id.isin(grouped_cc_df.loc[335749])]
display(duplicates)

print(f"\nDocument1\n----------\n{duplicates.iloc[0].text}")
print(f"\nDocument2\n----------\n{duplicates.iloc[1].text}")

,text,id,_curator_dedup_id,_duplicate_group_id
255162,"Once upon a time, there was a little girl name...",f03e9dfc-b61d-41a9-b816-cb99d3f77f3a,843160,335749
32337,"Once upon a time, there was a little girl name...",a56ba800-0f22-4770-a2b6-19c39e8e5561,99566,335749
170410,"Once upon a time, there was a little girl name...",2db7d179-9359-4c5a-ab5e-8112c9b11564,561484,335749



Document1
----------
Once upon a time, there was a little girl named Lily. She loved to eat spaghetti for dinner. One day, her mom made spaghetti with harmless tomato sauce. Lily was very happy and ate it all up. 

But then, her little brother came and spilled his juice all over the table. Lily sighed and her mom cleaned up the mess. After that, Lily played with her toys and went to bed. 

The next day, Lily's mom made spaghetti again. But this time, Lily didn't want to eat it. She sighed and said she wanted something else. Her mom made her a sandwich instead. The end.

Document2
----------
Once upon a time, there was a little girl named Lily. She loved to eat spaghetti for dinner. One day, her mom made spaghetti with harmless tomato sauce. Lily was very happy and ate it all up. 

But then, her little brother came and spilled his juice all over the table. Lily sighed and her mom cleaned up the mess. After that, Lily played with her toys and went to bed. 

The next day, Lily's mom made

In [14]:
client.stop()